# Image Transformations
Explore geometric transformations in OpenCV: translation, rotation, scaling, and perspective warps.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

def load_demo_image(path="../images/sample.jpg"):
    image = cv2.imread(path)
    if image is None:
        canvas = np.zeros((256, 256, 3), dtype=np.uint8)
        cv2.circle(canvas, (128, 128), 60, (0, 255, 0), 3)
        cv2.line(canvas, (30, 30), (220, 220), (255, 0, 0), 2)
        image = canvas
    return image

def show_bgr(img, title):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis("off")

image = load_demo_image()
show_bgr(image, "Demo Image")

## Translation
Shifting an image is done with an affine transform. Positive values move right/down.

In [ ]:
image = load_demo_image()
height, width = image.shape[:2]
translation_matrix = np.float32([[1, 0, 40], [0, 1, 60]])
translated = cv2.warpAffine(image, translation_matrix, (width, height))

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
show_bgr(image, f"Original ({width}x{height})")
plt.subplot(1, 2, 2)
show_bgr(translated, "Translated (+40, +60)")
plt.tight_layout()

## Question
How would you translate an image 50 pixels to the left and 30 pixels down? Why is the affine transform matrix defined as it is? Why is it 3x2 and not 2x2?

## Rotation
Rotation matrix uses the image center, angle in degrees, and a scaling factor.

In [ ]:
center = (width // 2, height // 2)

rotation_matrix = cv2.getRotationMatrix2D(center, angle=30, scale=1.0)

rotated = cv2.warpAffine(image, rotation_matrix, (width, height))
    
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)

show_bgr(image, "Original")

plt.subplot(1, 2, 2)

show_bgr(rotated, "Rotated 30 degrees")

plt.tight_layout()


## Question
How does the rotation matrix look for this rotation? Add a print and explain how it is constructed.

## Scaling
Use `cv2.resize` to scale up or down. Interpolation method controls quality.

In [ ]:
scaled_up = cv2.resize(image, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)

scaled_down = cv2.resize(image, None, fx=0.4, fy=0.4, interpolation=cv2.INTER_AREA)



plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)

show_bgr(image, f"Original ({width}x{height})")

plt.subplot(1, 3, 2)

show_bgr(scaled_up, f"Scaled to ({scaled_up.shape[1]}x{scaled_up.shape[0]}) (cubic)")

plt.subplot(1, 3, 3)

show_bgr(scaled_down, f"Scaled to ({scaled_down.shape[1]}x{scaled_down.shape[0]}) (area)")
plt.tight_layout()


## Question
Why is an interpolation method needed when scaling images?

## Perspective Transform
Map four source points to a new quadrilateral to correct perspective or create dramatic effects.

In [ ]:
source = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])
margin = 30
destination = np.float32([[margin, 0], [width - margin, 40], [width - margin, height - 40], [margin, height]])
perspective_matrix = cv2.getPerspectiveTransform(source, destination)
warped = cv2.warpPerspective(image, perspective_matrix, (width, height))

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
show_bgr(image, "Original")
plt.subplot(1, 2, 2)
show_bgr(warped, "Perspective warp")
plt.tight_layout()

## Question
What happens if the margin value is increased or decreased? Try different values and observe the effect on the warped image. What happens when the margin is larger than half the image width?

## Task
Implement a 3D rotation effect to flip the image around its central y axis using perspective transforms. You can visualize this by using the matplotlib animation module. Use the code from above to modify the code below to create this effect.

In [ ]:
from matplotlib import animation

image = load_demo_image()
height, width = image.shape[:2]
step = 10

fig, ax = plt.subplots()
#ax.imshow(image)

def animate(i):
    center = (width // 2, height // 2)

    rotation_matrix = cv2.getRotationMatrix2D(center, angle=i*step, scale=1.0)

    rotated = cv2.warpAffine(image, rotation_matrix, (width, height))
    rotated = cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB)
    ax.imshow(rotated)

ani = animation.FuncAnimation(fig, animate, frames=360//step + 1)

from IPython.display import HTML
HTML(ani.to_jshtml())